## 准备数据

In [38]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [39]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [40]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 256]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([256]), trainable=True)
        self.W2 = tf.Variable(tf.random.normal([256, 128]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([128]), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x= tf.reshape(x, [-1, 784])
        logits = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)    
        logits = tf.nn.relu(tf.matmul(logits, self.W2) + self.b2)
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [41]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [42]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 226.20683 ; accuracy 0.01005
epoch 1 : loss 215.8148 ; accuracy 0.013866667
epoch 2 : loss 207.68785 ; accuracy 0.01795
epoch 3 : loss 200.81358 ; accuracy 0.022416666
epoch 4 : loss 194.722 ; accuracy 0.026366666
epoch 5 : loss 189.1359 ; accuracy 0.029866667
epoch 6 : loss 183.90962 ; accuracy 0.033433333
epoch 7 : loss 178.94884 ; accuracy 0.0369
epoch 8 : loss 174.19646 ; accuracy 0.0402
epoch 9 : loss 169.62195 ; accuracy 0.0434
epoch 10 : loss 165.2196 ; accuracy 0.047333334
epoch 11 : loss 160.99622 ; accuracy 0.0518
epoch 12 : loss 156.95636 ; accuracy 0.057116665
epoch 13 : loss 153.12013 ; accuracy 0.062816665
epoch 14 : loss 149.49225 ; accuracy 0.068783335
epoch 15 : loss 146.04982 ; accuracy 0.07535
epoch 16 : loss 142.75943 ; accuracy 0.08155
epoch 17 : loss 139.57869 ; accuracy 0.08788333
epoch 18 : loss 136.46552 ; accuracy 0.09431667
epoch 19 : loss 133.38078 ; accuracy 0.100033335
epoch 20 : loss 130.29137 ; accuracy 0.10525
epoch 21 : loss 127.17375 ; 